In [1]:
import torch
import torch.autograd as autograd         # computation graph
from torch import Tensor                  # tensor node in the computation graph
import torch.nn as nn                     # neural networks
import torch.optim as optim               # optimizers e.g. gradient descent, ADAM, etc.

import matplotlib.pyplot as plt
import matplotlib.gridspec as gridspec
from mpl_toolkits.axes_grid1 import make_axes_locatable
from mpl_toolkits.mplot3d import Axes3D
import matplotlib.ticker
from torch.nn.parameter import Parameter

import numpy as np
import time
from pyDOE import lhs         #Latin Hypercube Sampling
import scipy.io

from smt.sampling_methods import LHS
from scipy.io import savemat

#Set default dtype to float32
torch.set_default_dtype(torch.float)

#PyTorch random number generator
torch.manual_seed(1234)

# Random number generators in other libraries
np.random.seed(1234)

# Device configuration
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')

print(device)

if device == 'cuda': 
    print(torch.cuda.get_device_name())

    

cuda:0


In [2]:
def true_1D_1(x): #True function for 1D_1 dy2/dx2 + dy/dx - 6y = 0; BC1: y(0)=2; BC2: dy/dx at (x=0) = -1;
    y = np.exp(2*x) + np.exp(-3*x)
    return y
    

In [3]:
label = "1D_SODE_tanh"
loss_thresh = 0.005

x = np.linspace(0,2,100).reshape(-1,1)

bc1_x = x[0].reshape(-1,1)
bc1_y = true_1D_1(x[0]).reshape(-1,1)
x_bc1_train = torch.from_numpy(bc1_x).float().to(device)
y_bc1_train = torch.from_numpy(bc1_y).float().to(device)
    

bc2_x = x[0].reshape(-1,1)
x_bc2_train = torch.from_numpy(bc2_x).float().to(device)
bc2_val = torch.tensor(-1.0,device=device)
bc2_val = bc2_val.view(1,1)

x_test = x.reshape(-1,1)
x_test_tensor = torch.from_numpy(x_test).float().to(device)
y_true = true_1D_1(x_test)
y_true_norm = np.linalg.norm(y_true,2)

# Domain bounds
lb = np.array(x[0]) 
ub = np.array(x[-1]) 

In [4]:
def colloc_pts(N_f,seed):
    #Collocation Points
    # Latin Hypercube sampling for collocation points 
    # N_f sets of tuples(x,y)
    x01 = np.array([[0.0, 1.0]])
    sampling = LHS(xlimits=x01,random_state =seed)
    
    x_coll_train = lb + (ub-lb)*sampling(N_f)
    x_coll_train = np.vstack((x_coll_train, bc1_x.reshape(-1,1))) # append training points to collocation points 

    return x_coll_train

In [5]:
class Sequentialmodel(nn.Module):
    
    def __init__(self,layers):
        super().__init__() #call __init__ from parent class 
              
    
        self.activation = nn.Tanh()
        self.loss_function = nn.MSELoss(reduction ='mean')
        
        'Initialise neural network as a list using nn.Modulelist'  
        self.linears = nn.ModuleList([nn.Linear(layers[i], layers[i+1]) for i in range(len(layers)-1)])
        
        for i in range(len(layers)-1):
            nn.init.xavier_normal_(self.linears[i].weight.data, gain=1.0)
            # set biases to zero
            nn.init.zeros_(self.linears[i].bias.data) 

    'forward pass'
    def forward(self,x):
        if torch.is_tensor(x) != True:         
            x = torch.from_numpy(x)                
        
        u_b = torch.from_numpy(ub).float().to(device)
        l_b = torch.from_numpy(lb).float().to(device)
                      
        #preprocessing input 
        x = (x - l_b)/(u_b - l_b) #feature scaling
        
        #convert to float
        a = x.float()
        
        for i in range(len(layers)-2):
            z = self.linears[i](a)
            a = self.activation(z)
            
        a = self.linears[-1](a) 
         
        return a
                        
    def loss_BC1(self,x,y):
                
        loss_bc1 = self.loss_function(self.forward(x), y)
                
        return loss_bc1
    
    def loss_BC2(self,x_bc2,bc2_val):
        g = x_bc2.clone()             
        g.requires_grad = True
        y = self.forward(g)    
            
        y_x = autograd.grad(y,g,torch.ones([x_bc2.shape[0], 1]).to(device), retain_graph=True, create_graph=True,allow_unused = True)[0]
        
        dy_dx = y_x[:,[0]]
        
        bc2 = dy_dx
        
        loss_bc2= self.loss_function(bc2,bc2_val)

        return loss_bc2
    
    def loss_PDE(self, x_coll,f_hat):
             
        g = x_coll.clone()             
        g.requires_grad = True
  
        y = self.forward(g) 

        y_x = autograd.grad(y,g,torch.ones([x_coll.shape[0], 1]).to(device), retain_graph=True, create_graph=True,allow_unused = True)[0]
        y_xx = autograd.grad(y_x,g,torch.ones(x_coll.shape).to(device), create_graph=True,allow_unused = True)[0]

        dy_dx = y_x[:,[0]]
        
        dy2_d2x = y_xx[:,[0]]
        
        f = dy2_d2x + dy_dx - 6*y
        
        loss_f = self.loss_function(f,f_hat)
                
        return loss_f
    
    
    def loss(self,x_bc1,y_bc1,x_bc2,bc2_val,x_coll,f_hat):

        loss_bc1 = self.loss_BC1(x_bc1,y_bc1)
        loss_bc2 = self.loss_BC2(x_bc2,bc2_val)
        loss_f = self.loss_PDE(x_coll,f_hat)
        
        loss_val = loss_bc1 + loss_bc2 + loss_f
        
        return loss_val
          
    'test neural network'
    
    def test(self):
        y_pred = self.forward(x_test_tensor)
        y_pred = y_pred.cpu().detach().numpy()

        return y_pred
    
    def test_loss(self):
        y_pred = self.test()
        
        test_mse = np.mean(np.square(y_pred.reshape(-1,1) - y_true.reshape(-1,1)))
        test_re = np.linalg.norm(y_pred.reshape(-1,1) - y_true.reshape(-1,1),2)/y_true_norm
        
        return test_mse, test_re

In [6]:
def train_step(x_coll,f_hat):
    def closure():
        optimizer.zero_grad()
        loss = PINN.loss(x_bc1_train,y_bc1_train,x_bc2_train,bc2_val,x_coll,f_hat)
        loss.backward()
        
        return loss

    optimizer.step(closure)

In [7]:
def data_update(loss_np):
    train_loss.append(loss_np)

    
    test_mse, test_re = PINN.test_loss()
    test_mse_loss.append(test_mse)
    test_re_loss.append(test_re)

In [8]:
def train_model(max_iter,rep):
    print(rep) 
    torch.manual_seed(rep*123)
    start_time = time.time()
    thresh_flag = 0

    for i in range(max_iter):
        x_coll = torch.from_numpy(colloc_pts(N_f,i*11)).float().to(device)
        f_hat = torch.zeros(x_coll.shape[0],1).to(device)
        train_step(x_coll,f_hat)
        
        loss_np = PINN.loss(x_bc1_train,y_bc1_train,x_bc2_train,bc2_val,x_coll,f_hat).cpu().detach().numpy()
        if(thresh_flag == 0):
            if(loss_np < loss_thresh):
                time_threshold[rep] = time.time() - start_time
                epoch_threshold[rep] = i+1            
                thresh_flag = 1       
        data_update(loss_np)
        print(i,"Train Loss",train_loss[-1],"Test MSE",test_mse_loss[-1],"Test RE",test_re_loss[-1])
    
    elapsed_time[rep] = time.time() - start_time
    print('Training time: %.2f' % (elapsed_time[rep]))

In [9]:
max_reps = 10
max_iter = 100

N_f = 1000

train_loss_full = []
test_mse_full = []
test_re_full = []
elapsed_time= np.zeros((max_reps,1))

time_threshold = np.empty((max_reps,1))
time_threshold[:] = np.nan
epoch_threshold = max_iter*np.ones((max_reps,1))

for reps in range(max_reps):
    
    train_loss = []
    test_mse_loss = []
    test_re_loss =[]
    
    'Generate Training data'
    torch.manual_seed(reps*36)
     #Total number of collocation points 
    
    
    layers = np.array([1,50,50,50,50,50,50,50,50,50,1]) #9 hidden layers
    PINN = Sequentialmodel(layers)
    PINN.to(device)

    'Neural Network Summary'
    print(PINN)

    params = list(PINN.parameters())
    
    optimizer = torch.optim.LBFGS(PINN.parameters(), lr=0.25, 
                              max_iter = 10, 
                              max_eval = 15, 
                              tolerance_grad = 1e-5, 
                              tolerance_change = 1e-5, 
                              history_size = 100, 
                              line_search_fn = 'strong_wolfe')
   

    start_time = time.time()
    train_model(max_iter,reps)

    
    torch.save(PINN.state_dict(),label+'_'+str(reps)+'.pt')
    train_loss_full.append(train_loss)
    test_mse_full.append(test_mse_loss)
    test_re_full.append(test_re_loss)
 
    
    print('Training time: %.2f' % (elapsed_time[reps]))

mdic = {"train_loss": train_loss_full,"test_mse_loss": test_mse_full, "test_re_loss": test_re_full, "Time": elapsed_time, "label": label, "Thresh Time": time_threshold,"Thresh epoch": epoch_threshold}
savemat(label+'.mat', mdic)

Sequentialmodel(
  (activation): Tanh()
  (loss_function): MSELoss()
  (linears): ModuleList(
    (0): Linear(in_features=1, out_features=50, bias=True)
    (1): Linear(in_features=50, out_features=50, bias=True)
    (2): Linear(in_features=50, out_features=50, bias=True)
    (3): Linear(in_features=50, out_features=50, bias=True)
    (4): Linear(in_features=50, out_features=50, bias=True)
    (5): Linear(in_features=50, out_features=50, bias=True)
    (6): Linear(in_features=50, out_features=50, bias=True)
    (7): Linear(in_features=50, out_features=50, bias=True)
    (8): Linear(in_features=50, out_features=50, bias=True)
    (9): Linear(in_features=50, out_features=1, bias=True)
  )
)
0
0 Train Loss 4.3680706 Test MSE 386.0759172229266 Test RE 1.001610661044255
1 Train Loss 3.7148273 Test MSE 386.52098520649474 Test RE 1.0021878227063223
2 Train Loss 3.4524229 Test MSE 385.40664717340957 Test RE 1.0007421287761424
3 Train Loss 2.4228966 Test MSE 385.034690202943 Test RE 1.000259102

97 Train Loss 2.3810344 Test MSE 383.80891734084855 Test RE 0.9986656517446741
98 Train Loss 2.3810344 Test MSE 383.80891734084855 Test RE 0.9986656517446741
99 Train Loss 2.3810344 Test MSE 383.80891734084855 Test RE 0.9986656517446741
Training time: 22.36
Training time: 22.36
Sequentialmodel(
  (activation): Tanh()
  (loss_function): MSELoss()
  (linears): ModuleList(
    (0): Linear(in_features=1, out_features=50, bias=True)
    (1): Linear(in_features=50, out_features=50, bias=True)
    (2): Linear(in_features=50, out_features=50, bias=True)
    (3): Linear(in_features=50, out_features=50, bias=True)
    (4): Linear(in_features=50, out_features=50, bias=True)
    (5): Linear(in_features=50, out_features=50, bias=True)
    (6): Linear(in_features=50, out_features=50, bias=True)
    (7): Linear(in_features=50, out_features=50, bias=True)
    (8): Linear(in_features=50, out_features=50, bias=True)
    (9): Linear(in_features=50, out_features=1, bias=True)
  )
)
1
0 Train Loss 4.345121

93 Train Loss 2.3812656 Test MSE 383.66502013975037 Test RE 0.9984784249013179
94 Train Loss 2.3812656 Test MSE 383.66502013975037 Test RE 0.9984784249013179
95 Train Loss 2.3812656 Test MSE 383.66502013975037 Test RE 0.9984784249013179
96 Train Loss 2.3812656 Test MSE 383.66502013975037 Test RE 0.9984784249013179
97 Train Loss 2.3812656 Test MSE 383.66502013975037 Test RE 0.9984784249013179
98 Train Loss 2.3812656 Test MSE 383.66502013975037 Test RE 0.9984784249013179
99 Train Loss 2.3812656 Test MSE 383.66502013975037 Test RE 0.9984784249013179
Training time: 8.33
Training time: 8.33
Sequentialmodel(
  (activation): Tanh()
  (loss_function): MSELoss()
  (linears): ModuleList(
    (0): Linear(in_features=1, out_features=50, bias=True)
    (1): Linear(in_features=50, out_features=50, bias=True)
    (2): Linear(in_features=50, out_features=50, bias=True)
    (3): Linear(in_features=50, out_features=50, bias=True)
    (4): Linear(in_features=50, out_features=50, bias=True)
    (5): Linea

88 Train Loss 2.3809586 Test MSE 383.76340881050766 Test RE 0.9986064436969516
89 Train Loss 2.3809586 Test MSE 383.76340881050766 Test RE 0.9986064436969516
90 Train Loss 2.3809586 Test MSE 383.76340881050766 Test RE 0.9986064436969516
91 Train Loss 2.3809586 Test MSE 383.76340881050766 Test RE 0.9986064436969516
92 Train Loss 2.3809586 Test MSE 383.76340881050766 Test RE 0.9986064436969516
93 Train Loss 2.3809586 Test MSE 383.76340881050766 Test RE 0.9986064436969516
94 Train Loss 2.3809586 Test MSE 383.76340881050766 Test RE 0.9986064436969516
95 Train Loss 2.3809586 Test MSE 383.76340881050766 Test RE 0.9986064436969516
96 Train Loss 2.3809586 Test MSE 383.76340881050766 Test RE 0.9986064436969516
97 Train Loss 2.3809586 Test MSE 383.76340881050766 Test RE 0.9986064436969516
98 Train Loss 2.3809586 Test MSE 383.76340881050766 Test RE 0.9986064436969516
99 Train Loss 2.3809586 Test MSE 383.76340881050766 Test RE 0.9986064436969516
Training time: 9.62
Training time: 9.62
Sequentialmo

86 Train Loss 2.3806286 Test MSE 383.6304211318412 Test RE 0.9984334023686505
87 Train Loss 2.3806286 Test MSE 383.6304211318412 Test RE 0.9984334023686505
88 Train Loss 2.3806286 Test MSE 383.6304211318412 Test RE 0.9984334023686505
89 Train Loss 2.3806286 Test MSE 383.6304211318412 Test RE 0.9984334023686505
90 Train Loss 2.3806286 Test MSE 383.6304211318412 Test RE 0.9984334023686505
91 Train Loss 2.3806286 Test MSE 383.6304211318412 Test RE 0.9984334023686505
92 Train Loss 2.3806286 Test MSE 383.6304211318412 Test RE 0.9984334023686505
93 Train Loss 2.3806286 Test MSE 383.6304211318412 Test RE 0.9984334023686505
94 Train Loss 2.3806286 Test MSE 383.6304211318412 Test RE 0.9984334023686505
95 Train Loss 2.3806286 Test MSE 383.6304211318412 Test RE 0.9984334023686505
96 Train Loss 2.3806286 Test MSE 383.6304211318412 Test RE 0.9984334023686505
97 Train Loss 2.3806286 Test MSE 383.6304211318412 Test RE 0.9984334023686505
98 Train Loss 2.3806286 Test MSE 383.6304211318412 Test RE 0.998

...

80 Train Loss 2.3809838 Test MSE 383.75170459289797 Test RE 0.9985912155694483
81 Train Loss 2.3809838 Test MSE 383.75170459289797 Test RE 0.9985912155694483
82 Train Loss 2.3809838 Test MSE 383.75170459289797 Test RE 0.9985912155694483
83 Train Loss 2.3809838 Test MSE 383.75170459289797 Test RE 0.9985912155694483
84 Train Loss 2.3809838 Test MSE 383.75170459289797 Test RE 0.9985912155694483
85 Train Loss 2.3809838 Test MSE 383.75170459289797 Test RE 0.9985912155694483
86 Train Loss 2.3809838 Test MSE 383.75170459289797 Test RE 0.9985912155694483
87 Train Loss 2.3809838 Test MSE 383.75170459289797 Test RE 0.9985912155694483
88 Train Loss 2.3809838 Test MSE 383.75170459289797 Test RE 0.9985912155694483
89 Train Loss 2.3809838 Test MSE 383.75170459289797 Test RE 0.9985912155694483
90 Train Loss 2.3809838 Test MSE 383.75170459289797 Test RE 0.9985912155694483
91 Train Loss 2.3809838 Test MSE 383.75170459289797 Test RE 0.9985912155694483
92 Train Loss 2.3809838 Test MSE 383.75170459289797 

76 Train Loss 2.381093 Test MSE 383.76977958913403 Test RE 0.9986147324942141
77 Train Loss 2.381093 Test MSE 383.76977958913403 Test RE 0.9986147324942141
78 Train Loss 2.381093 Test MSE 383.76977958913403 Test RE 0.9986147324942141
79 Train Loss 2.381093 Test MSE 383.76977958913403 Test RE 0.9986147324942141
80 Train Loss 2.381093 Test MSE 383.76977958913403 Test RE 0.9986147324942141
81 Train Loss 2.381093 Test MSE 383.76977958913403 Test RE 0.9986147324942141
82 Train Loss 2.381093 Test MSE 383.76977958913403 Test RE 0.9986147324942141
83 Train Loss 2.381093 Test MSE 383.76977958913403 Test RE 0.9986147324942141
84 Train Loss 2.381093 Test MSE 383.76977958913403 Test RE 0.9986147324942141
85 Train Loss 2.381093 Test MSE 383.76977958913403 Test RE 0.9986147324942141
86 Train Loss 2.381093 Test MSE 383.76977958913403 Test RE 0.9986147324942141
87 Train Loss 2.381093 Test MSE 383.76977958913403 Test RE 0.9986147324942141
88 Train Loss 2.381093 Test MSE 383.76977958913403 Test RE 0.998

73 Train Loss 2.3812811 Test MSE 383.8425347910859 Test RE 0.9987093868641812
74 Train Loss 2.3812811 Test MSE 383.8425347910859 Test RE 0.9987093868641812
75 Train Loss 2.3812811 Test MSE 383.8425347910859 Test RE 0.9987093868641812
76 Train Loss 2.3812811 Test MSE 383.8425347910859 Test RE 0.9987093868641812
77 Train Loss 2.3812811 Test MSE 383.8425347910859 Test RE 0.9987093868641812
78 Train Loss 2.3812811 Test MSE 383.8425347910859 Test RE 0.9987093868641812
79 Train Loss 2.3812811 Test MSE 383.8425347910859 Test RE 0.9987093868641812
80 Train Loss 2.3812811 Test MSE 383.8425347910859 Test RE 0.9987093868641812
81 Train Loss 2.3812811 Test MSE 383.8425347910859 Test RE 0.9987093868641812
82 Train Loss 2.3812811 Test MSE 383.8425347910859 Test RE 0.9987093868641812
83 Train Loss 2.3812811 Test MSE 383.8425347910859 Test RE 0.9987093868641812
84 Train Loss 2.3812811 Test MSE 383.8425347910859 Test RE 0.9987093868641812
85 Train Loss 2.3812811 Test MSE 383.8425347910859 Test RE 0.998

69 Train Loss 2.3750992 Test MSE 382.3708464302168 Test RE 0.9967929753389498
70 Train Loss 2.3750954 Test MSE 382.37185325889084 Test RE 0.996794287676303
71 Train Loss 2.375093 Test MSE 382.37278742298156 Test RE 0.9967955052984327
72 Train Loss 2.3750865 Test MSE 382.37265347578744 Test RE 0.9967953307070692
73 Train Loss 2.3750768 Test MSE 382.3725479498041 Test RE 0.9967951931608641
74 Train Loss 2.3750672 Test MSE 382.37018178469725 Test RE 0.9967921090147007
75 Train Loss 2.373781 Test MSE 382.30208434882815 Test RE 0.9967033442359421
76 Train Loss 2.3718007 Test MSE 381.86089212527594 Test RE 0.9961280600238388
77 Train Loss 2.3689778 Test MSE 381.49301959588456 Test RE 0.9956481255041857
78 Train Loss 2.3627687 Test MSE 379.99622768784496 Test RE 0.9936929878790763
79 Train Loss 2.3621237 Test MSE 379.9301936956425 Test RE 0.9936066444351724
80 Train Loss 2.3507397 Test MSE 377.3100647327349 Test RE 0.9901745908892255
81 Train Loss 2.3323355 Test MSE 374.6859008380585 Test RE 

65 Train Loss 2.3811097 Test MSE 383.8184431749472 Test RE 0.9986780447140167
66 Train Loss 2.3811097 Test MSE 383.8184431749472 Test RE 0.9986780447140167
67 Train Loss 2.3811097 Test MSE 383.8184431749472 Test RE 0.9986780447140167
68 Train Loss 2.3811097 Test MSE 383.8184431749472 Test RE 0.9986780447140167
69 Train Loss 2.3811097 Test MSE 383.8184431749472 Test RE 0.9986780447140167
70 Train Loss 2.3811097 Test MSE 383.8184431749472 Test RE 0.9986780447140167
71 Train Loss 2.3811097 Test MSE 383.8184431749472 Test RE 0.9986780447140167
72 Train Loss 2.3811097 Test MSE 383.8184431749472 Test RE 0.9986780447140167
73 Train Loss 2.3811097 Test MSE 383.8184431749472 Test RE 0.9986780447140167
74 Train Loss 2.3811097 Test MSE 383.8184431749472 Test RE 0.9986780447140167
75 Train Loss 2.3811097 Test MSE 383.8184431749472 Test RE 0.9986780447140167
76 Train Loss 2.3811097 Test MSE 383.8184431749472 Test RE 0.9986780447140167
77 Train Loss 2.3811097 Test MSE 383.8184431749472 Test RE 0.998

In [10]:
a = 0
for i in range(10):
    a = a + test_re_full[i][-1]
print(a/10)

0.9746650559652632
